In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
df_agricultura = pd.read_csv('TabelaAgricultura.csv')
df_desastres = pd.read_csv('TabelaDesastres.csv')
df_desemprego = pd.read_csv('TabelaDesemprego.csv')
df_GDP = pd.read_csv('TabelaGDP.csv')
df_populacao = pd.read_csv('TabelaPopulacao.csv')
df_turismo = pd.read_csv('TabelaTurismo.csv')

C:\Users\franv\AppData\Local\Temp\ipykernel_12132\2087146932.py:2: DtypeWarning: Columns (23,24,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_desastres = pd.read_csv('TabelaDesastres.csv')


# Criação das Dimensões e Tabela de Factos

In [3]:
cols_to_convert = ['Start_year', 'Start_month', 'Start_day', 'End_year', 'End_month', 'End_day']
for col in cols_to_convert:
    df_desastres[col] = pd.to_numeric(df_desastres[col], errors='coerce')
    df_desastres[col] = df_desastres[col].replace([np.inf, -np.inf], np.nan) # replace inf values with NaN
    df_desastres[col] = df_desastres[col].astype('Int64') # convert to nullable integer type

df_desastres

,DisasterID,Year,Seq,Disaster_group,Disaster_subgroup,Disaster_type,Disaster_subtype,Disaster_subsubtype,Event_name,Country,...,N_affected,N_homeless,Total_affected,Reconstruction_costs,Reconstruction_costs_adjusted,Insured_damages,Insured_damages_adjusted,Total_damages,Total_damages_adjusted,CPI
0,1900-9002-CPV,1900,9002,Natural,Climatological,Drought,Drought,NaN,NaN,Cabo Verde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,849084409"
1,1900-9001-IND,1900,9001,Natural,Climatological,Drought,Drought,NaN,NaN,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,849084409"
2,1902-0012-GTM,1902,12,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Guatemala,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,843726.0,"2,963047785"
3,1902-0003-GTM,1902,3,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,963047785"
4,1902-0010-GTM,1902,10,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,963047785"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,2023-0080-ZAF,2023,80,Natural,Hydrological,Flood,NaN,NaN,NaN,South Africa,...,1000.0,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16564,2023-0110-ZMB,2023,110,Natural,Hydrological,Flood,Flash flood,NaN,NaN,Zambia,...,154608.0,NaN,154608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16565,2023-0068-ZMB,2023,68,Natural,Biological,Epidemic,Bacterial disease,NaN,Cholera,Zambia,...,NaN,NaN,118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16566,2023-0095-ZWE,2023,95,Natural,Meteorological,Storm,Tropical cyclone,NaN,Tropical cyclone 'Freddy',Zimbabwe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Split Local_time into Hour and Minutes columns
df_desastres[['Hour', 'Minutes']] = df_desastres['Local_time'].str.extract('(\d+):(\d+)')
df_desastres['Hour'] = pd.to_numeric(df_desastres['Hour'], errors='coerce')
df_desastres['Hour'] = df_desastres['Hour'].astype('Int64')
df_desastres['Minutes'] = pd.to_numeric(df_desastres['Minutes'], errors='coerce')
df_desastres['Minutes'] = df_desastres['Minutes'].astype('Int64')

In [5]:
def Semester(month):
    if pd.isna(month):
        return np.nan
    elif month < 6:
        return "1st Semester"
    elif month >= 6 and month < 13:
        return "2nd Semester"

def Quarter(month):
    if pd.isna(month):
        return np.nan
    elif month < 3:
        return "1st Quarter"
    elif month >= 3 and month < 6:
        return "2nd Quarter"
    elif month >= 6 and month < 9:
        return "3rd Quarter"
    elif month >= 9 and month <13:
        return "4th Quarter"
    

def Weekday(date):
    if pd.isna(date):
        return np.nan
    try:
        weekday_name = date.strftime('%A')
    except AttributeError:
        return None
    return weekday_name

def Season(date):
    if date is None:
        return None
    try:
        month = date.month
        day = date.day
    except AttributeError:
        return nan
    if (month == 3 and day >= 20) or month in [4, 5, 6] or (month == 7 and day <= 20):
        return 'Spring'
    elif (month == 6 and day >= 21) or month in [7, 8, 9] or (month == 10 and day <= 21):
        return 'Summer'
    elif (month == 9 and day >= 22) or month in [10, 11, 12] or (month == 12 and day <= 20):
        return 'Fall'
    elif (month == 12 and day >= 21) or month in [1, 2, 3] or (month == 3 and day <= 19):
        return 'Winter'
    else:
        return np.nan

def Decade(year):
    return year[0:3] + "0s"

In [6]:
# Check for null values in Year, Month, and Day columns
mask = df_desastres[['Start_year', 'Start_month', 'Start_day']].isnull().any(axis=1)

# Create New Date column with null values
df_desastres['Start_Date'] = pd.NaT

# Define Start_Date as DateTime
df_desastres.loc[~mask, 'Start_Date'] = pd.to_datetime(df_desastres.loc[~mask, ['Start_day', 'Start_month', 'Start_year']].astype(str).agg('-'.join, axis=1), format='%d-%m-%Y', errors='coerce')
df_desastres['Start_Date'] = df_desastres['Start_Date'].dt.date

# Check for null values in Year, Month, and Day columns
mask = df_desastres[['End_year', 'End_month', 'End_day']].isnull().any(axis=1)

# Create New Date column with null values
df_desastres['End_Date'] = pd.NaT
df_desastres.loc[~mask, 'End_Date'] = pd.to_datetime(df_desastres.loc[~mask, ['End_day', 'End_month', 'End_year']].astype(str).agg('-'.join, axis=1), format='%d-%m-%Y', errors='coerce')
df_desastres['End_Date'] = df_desastres['End_Date'].dt.date

In [7]:
# Add the columns to the dataframe
df_desastres['Start_semester'] = (df_desastres['Start_month']).apply(Semester)
df_desastres['Start_quarter'] = (df_desastres['Start_month']).apply(Quarter)
df_desastres['Start_weekday'] = pd.to_datetime(df_desastres['Start_Date'], errors='coerce').apply(Weekday)
df_desastres['Start_season'] = pd.to_datetime(df_desastres['Start_Date'], errors='coerce').apply(Season)
df_desastres['Start_decade'] = pd.to_datetime(df_desastres['Start_year'], format='%Y', errors='coerce').dt.strftime('%Y').apply(Decade)

df_desastres['End_semester'] = (df_desastres['End_month']).apply(Semester)
df_desastres['End_quarter'] = (df_desastres['End_month']).apply(Quarter)
df_desastres['End_weekday'] = pd.to_datetime(df_desastres['End_Date'], errors='coerce').apply(Weekday)
df_desastres['End_season'] = pd.to_datetime(df_desastres['End_Date'], errors='coerce').apply(Season)
df_desastres['End_decade'] = pd.to_datetime(df_desastres['End_year'], format='%Y', errors='coerce').dt.strftime('%Y').apply(Decade)

df_desastres.replace(pd.NaT, np.nan, inplace=True)


In [8]:
df_desastres[['Local_time','Hour', 'Minutes', 'Start_year', 'Start_month', 'Start_day', 'Start_Date',
                'Start_semester', 'Start_quarter', 'Start_weekday', 'Start_season', 'Start_decade',  
                'End_year', 'End_month', 'End_day', 'End_Date',
                'End_semester', 'End_quarter', 'End_weekday', 'End_season', 'End_decade']]

,Local_time,Hour,Minutes,Start_year,Start_month,Start_day,Start_Date,Start_semester,Start_quarter,Start_weekday,...,Start_decade,End_year,End_month,End_day,End_Date,End_semester,End_quarter,End_weekday,End_season,End_decade
0,NaN,<NA>,<NA>,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,...,1900s,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,1900s
1,NaN,<NA>,<NA>,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,...,1900s,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,1900s
2,20:20,20,20,1902,4,18,1902-04-18,1st Semester,2nd Quarter,Friday,...,1900s,1902,4,18,1902-04-18,1st Semester,2nd Quarter,Friday,Spring,1900s
3,NaN,<NA>,<NA>,1902,4,8,1902-04-08,1st Semester,2nd Quarter,Tuesday,...,1900s,1902,4,8,1902-04-08,1st Semester,2nd Quarter,Tuesday,Spring,1900s
4,NaN,<NA>,<NA>,1902,10,24,1902-10-24,2nd Semester,4th Quarter,Friday,...,1900s,1902,10,24,1902-10-24,2nd Semester,4th Quarter,Friday,Fall,1900s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,NaN,<NA>,<NA>,2023,2,8,2023-02-08,1st Semester,1st Quarter,Wednesday,...,2020s,2023,2,21,2023-02-21,1st Semester,1st Quarter,Tuesday,Winter,2020s
16564,NaN,<NA>,<NA>,2023,2,7,2023-02-07,1st Semester,1st Quarter,Tuesday,...,2020s,2023,3,9,2023-03-09,1st Semester,2nd Quarter,Thursday,Winter,2020s
16565,NaN,<NA>,<NA>,2023,1,21,2023-01-21,1st Semester,1st Quarter,Saturday,...,2020s,2023,1,30,2023-01-30,1st Semester,1st Quarter,Monday,Winter,2020s
16566,NaN,<NA>,<NA>,2023,2,24,2023-02-24,1st Semester,1st Quarter,Friday,...,2020s,2023,3,8,2023-03-08,1st Semester,2nd Quarter,Wednesday,Winter,2020s


In [9]:
df_desastres

,DisasterID,Year,Seq,Disaster_group,Disaster_subgroup,Disaster_type,Disaster_subtype,Disaster_subsubtype,Event_name,Country,...,Start_semester,Start_quarter,Start_weekday,Start_season,Start_decade,End_semester,End_quarter,End_weekday,End_season,End_decade
0,1900-9002-CPV,1900,9002,Natural,Climatological,Drought,Drought,NaN,NaN,Cabo Verde,...,NaN,NaN,NaN,NaN,1900s,NaN,NaN,NaN,NaN,1900s
1,1900-9001-IND,1900,9001,Natural,Climatological,Drought,Drought,NaN,NaN,India,...,NaN,NaN,NaN,NaN,1900s,NaN,NaN,NaN,NaN,1900s
2,1902-0012-GTM,1902,12,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Guatemala,...,1st Semester,2nd Quarter,Friday,Spring,1900s,1st Semester,2nd Quarter,Friday,Spring,1900s
3,1902-0003-GTM,1902,3,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,1st Semester,2nd Quarter,Tuesday,Spring,1900s,1st Semester,2nd Quarter,Tuesday,Spring,1900s
4,1902-0010-GTM,1902,10,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,2nd Semester,4th Quarter,Friday,Fall,1900s,2nd Semester,4th Quarter,Friday,Fall,1900s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,2023-0080-ZAF,2023,80,Natural,Hydrological,Flood,NaN,NaN,NaN,South Africa,...,1st Semester,1st Quarter,Wednesday,Winter,2020s,1st Semester,1st Quarter,Tuesday,Winter,2020s
16564,2023-0110-ZMB,2023,110,Natural,Hydrological,Flood,Flash flood,NaN,NaN,Zambia,...,1st Semester,1st Quarter,Tuesday,Winter,2020s,1st Semester,2nd Quarter,Thursday,Winter,2020s
16565,2023-0068-ZMB,2023,68,Natural,Biological,Epidemic,Bacterial disease,NaN,Cholera,Zambia,...,1st Semester,1st Quarter,Saturday,Winter,2020s,1st Semester,1st Quarter,Monday,Winter,2020s
16566,2023-0095-ZWE,2023,95,Natural,Meteorological,Storm,Tropical cyclone,NaN,Tropical cyclone 'Freddy',Zimbabwe,...,1st Semester,1st Quarter,Friday,Winter,2020s,1st Semester,2nd Quarter,Wednesday,Winter,2020s


# Dimensão Date

In [10]:
dimData = df_desastres[['Local_time','Hour', 'Minutes', 'Start_year', 'Start_month', 'Start_day', 'Start_Date',
                'Start_semester', 'Start_quarter', 'Start_weekday', 'Start_season', 'Start_decade',  
                'End_year', 'End_month', 'End_day', 'End_Date',
                'End_semester', 'End_quarter', 'End_weekday', 'End_season', 'End_decade']].drop_duplicates()

In [11]:
dimData.drop_duplicates()

,Local_time,Hour,Minutes,Start_year,Start_month,Start_day,Start_Date,Start_semester,Start_quarter,Start_weekday,...,Start_decade,End_year,End_month,End_day,End_Date,End_semester,End_quarter,End_weekday,End_season,End_decade
0,NaN,<NA>,<NA>,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,...,1900s,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,1900s
2,20:20,20,20,1902,4,18,1902-04-18,1st Semester,2nd Quarter,Friday,...,1900s,1902,4,18,1902-04-18,1st Semester,2nd Quarter,Friday,Spring,1900s
3,NaN,<NA>,<NA>,1902,4,8,1902-04-08,1st Semester,2nd Quarter,Tuesday,...,1900s,1902,4,8,1902-04-08,1st Semester,2nd Quarter,Tuesday,Spring,1900s
4,NaN,<NA>,<NA>,1902,10,24,1902-10-24,2nd Semester,4th Quarter,Friday,...,1900s,1902,10,24,1902-10-24,2nd Semester,4th Quarter,Friday,Fall,1900s
5,NaN,<NA>,<NA>,1903,4,29,1903-04-29,1st Semester,2nd Quarter,Wednesday,...,1900s,1903,4,29,1903-04-29,1st Semester,2nd Quarter,Wednesday,Spring,1900s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,NaN,<NA>,<NA>,2023,2,8,2023-02-08,1st Semester,1st Quarter,Wednesday,...,2020s,2023,2,21,2023-02-21,1st Semester,1st Quarter,Tuesday,Winter,2020s
16564,NaN,<NA>,<NA>,2023,2,7,2023-02-07,1st Semester,1st Quarter,Tuesday,...,2020s,2023,3,9,2023-03-09,1st Semester,2nd Quarter,Thursday,Winter,2020s
16565,NaN,<NA>,<NA>,2023,1,21,2023-01-21,1st Semester,1st Quarter,Saturday,...,2020s,2023,1,30,2023-01-30,1st Semester,1st Quarter,Monday,Winter,2020s
16566,NaN,<NA>,<NA>,2023,2,24,2023-02-24,1st Semester,1st Quarter,Friday,...,2020s,2023,3,8,2023-03-08,1st Semester,2nd Quarter,Wednesday,Winter,2020s


In [12]:
dimStartData = df_desastres[['Local_time','Hour', 'Minutes', 'Start_year', 'Start_month', 'Start_day', 'Start_Date',
                'Start_semester', 'Start_quarter', 'Start_weekday', 'Start_season', 'Start_decade']].drop_duplicates()
dimStartData.insert(0, 'StartDateKey', np.arange(len(dimStartData)))

dimEndData =  df_desastres[['End_year', 'End_month', 'End_day', 'End_Date',
                'End_semester', 'End_quarter', 'End_weekday', 'End_season', 'End_decade']].drop_duplicates()

#dimEndData insert missing columns with all null
dimEndData['Local_time'] = pd.Series(dtype=pd.Int64Dtype())
dimEndData['Hour'] = pd.Series(dtype=pd.Int64Dtype())
dimEndData['Minutes'] = pd.Series(dtype=pd.Int64Dtype())


dimEndData.insert(0, 'EndDateKey', np.arange(len(dimEndData)))

dimStartDataRenamed = dimStartData.copy()
dimStartDataRenamed.columns = ['StartDateKey', 'Local_time','Hour', 'Minutes', 'Year', 'Month', 'Day', 'Date',
                'Semester', 'Quarter', 'Weekday', 'Season', 'Decade']

dimStartDataRenamed.reset_index(drop=True, inplace=True)

dimEndDataRenamed = dimEndData.copy()
dimEndDataRenamed.columns = ['EndDateKey', 'Year', 'Month', 'Day', 'Date',
                'Semester', 'Quarter', 'Weekday', 'Season', 'Decade', 'Local_time','Hour', 'Minutes']

dimEndDataRenamed = dimEndDataRenamed.reindex(columns=['EndDateKey','Local_time','Hour', 'Minutes', 'Year', 'Month', 'Day', 'Date',
                'Semester', 'Quarter', 'Weekday', 'Season', 'Decade'])

dimEndDataRenamed.reset_index(drop=True, inplace=True)
dimEndDataRenamed['Local_time'] = dimEndDataRenamed['Local_time'].astype(str)

dimStartDataRenamed.drop_duplicates()
dimEndDataRenamed.drop_duplicates()

,EndDateKey,Local_time,Hour,Minutes,Year,Month,Day,Date,Semester,Quarter,Weekday,Season,Decade
0,0,<NA>,<NA>,<NA>,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,1900s
1,1,<NA>,<NA>,<NA>,1902,4,18,1902-04-18,1st Semester,2nd Quarter,Friday,Spring,1900s
2,2,<NA>,<NA>,<NA>,1902,4,8,1902-04-08,1st Semester,2nd Quarter,Tuesday,Spring,1900s
3,3,<NA>,<NA>,<NA>,1902,10,24,1902-10-24,2nd Semester,4th Quarter,Friday,Fall,1900s
4,4,<NA>,<NA>,<NA>,1903,4,29,1903-04-29,1st Semester,2nd Quarter,Wednesday,Spring,1900s
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9401,9401,<NA>,<NA>,<NA>,2023,2,24,2023-02-24,1st Semester,1st Quarter,Friday,Winter,2020s
9402,9402,<NA>,<NA>,<NA>,2023,2,27,2023-02-27,1st Semester,1st Quarter,Monday,Winter,2020s
9403,9403,<NA>,<NA>,<NA>,2023,1,12,2023-01-12,1st Semester,1st Quarter,Thursday,Winter,2020s
9404,9404,<NA>,<NA>,<NA>,2023,3,9,2023-03-09,1st Semester,2nd Quarter,Thursday,Winter,2020s


In [13]:
dimEndData.copy()

,EndDateKey,End_year,End_month,End_day,End_Date,End_semester,End_quarter,End_weekday,End_season,End_decade,Local_time,Hour,Minutes
0,0,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,1900s,<NA>,<NA>,<NA>
2,1,1902,4,18,1902-04-18,1st Semester,2nd Quarter,Friday,Spring,1900s,<NA>,<NA>,<NA>
3,2,1902,4,8,1902-04-08,1st Semester,2nd Quarter,Tuesday,Spring,1900s,<NA>,<NA>,<NA>
4,3,1902,10,24,1902-10-24,2nd Semester,4th Quarter,Friday,Fall,1900s,<NA>,<NA>,<NA>
5,4,1903,4,29,1903-04-29,1st Semester,2nd Quarter,Wednesday,Spring,1900s,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16558,9401,2023,2,24,2023-02-24,1st Semester,1st Quarter,Friday,Winter,2020s,<NA>,<NA>,<NA>
16560,9402,2023,2,27,2023-02-27,1st Semester,1st Quarter,Monday,Winter,2020s,<NA>,<NA>,<NA>
16562,9403,2023,1,12,2023-01-12,1st Semester,1st Quarter,Thursday,Winter,2020s,<NA>,<NA>,<NA>
16564,9404,2023,3,9,2023-03-09,1st Semester,2nd Quarter,Thursday,Winter,2020s,<NA>,<NA>,<NA>


In [14]:
# Concatenate the rows from both dataframes
dimData = pd.concat([dimStartDataRenamed[['Local_time','Hour', 'Minutes', 'Year', 'Month', 'Day', 'Date', 'Semester', 'Quarter', 'Weekday', 'Season', 'Decade']],
                 dimEndDataRenamed[['Local_time','Hour', 'Minutes', 'Year', 'Month', 'Day', 'Date', 'Semester', 'Quarter', 'Weekday', 'Season', 'Decade']]])
dimData.insert(0, 'DateKey', np.arange(len(dimData)))
dimData

,DateKey,Local_time,Hour,Minutes,Year,Month,Day,Date,Semester,Quarter,Weekday,Season,Decade
0,0,NaN,<NA>,<NA>,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,1900s
1,1,20:20,20,20,1902,4,18,1902-04-18,1st Semester,2nd Quarter,Friday,Spring,1900s
2,2,NaN,<NA>,<NA>,1902,4,8,1902-04-08,1st Semester,2nd Quarter,Tuesday,Spring,1900s
3,3,NaN,<NA>,<NA>,1902,10,24,1902-10-24,2nd Semester,4th Quarter,Friday,Fall,1900s
4,4,NaN,<NA>,<NA>,1903,4,29,1903-04-29,1st Semester,2nd Quarter,Wednesday,Spring,1900s
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9401,19216,<NA>,<NA>,<NA>,2023,2,24,2023-02-24,1st Semester,1st Quarter,Friday,Winter,2020s
9402,19217,<NA>,<NA>,<NA>,2023,2,27,2023-02-27,1st Semester,1st Quarter,Monday,Winter,2020s
9403,19218,<NA>,<NA>,<NA>,2023,1,12,2023-01-12,1st Semester,1st Quarter,Thursday,Winter,2020s
9404,19219,<NA>,<NA>,<NA>,2023,3,9,2023-03-09,1st Semester,2nd Quarter,Thursday,Winter,2020s


In [15]:
df_desastres = pd.merge(df_desastres, dimStartData, on=['Local_time','Hour', 'Minutes', 'Start_year', 'Start_month', 'Start_day', 'Start_Date',
                'Start_semester', 'Start_quarter', 'Start_weekday', 'Start_season', 'Start_decade'], how='left')
df_desastres = df_desastres.drop(['Local_time','Hour', 'Minutes', 'Start_year', 'Start_month', 'Start_day', 'Start_Date',
                'Start_semester', 'Start_quarter', 'Start_weekday', 'Start_season', 'Start_decade'], axis=1)


df_desastres = pd.merge(df_desastres, dimEndData, on=['End_year', 'End_month', 'End_day', 'End_Date',
                'End_semester', 'End_quarter', 'End_weekday', 'End_season', 'End_decade'], how='left')
df_desastres = df_desastres.drop(['End_year', 'End_month', 'End_day', 'End_Date',
                'End_semester', 'End_quarter', 'End_weekday', 'End_season', 'End_decade', 'Local_time','Hour', 'Minutes'], axis=1)


In [16]:
dimData['Year'] = dimData['Year'].astype('Int64').fillna(pd.NA)
dimData['Month'] = dimData['Month'].astype('Int64').fillna(pd.NA)
dimData['Day'] = dimData['Day'].astype('Int64').fillna(pd.NA)

In [17]:
dimEndDataRenamed['Local_time'] = dimEndDataRenamed['Local_time'].astype(str)

In [18]:
cols_to_compare = ['Local_time', 'Hour', 'Minutes', 'Year', 'Month', 'Day', 'Date', 'Semester', 'Quarter', 'Weekday', 'Season', 'Decade']
matching_ids_start = []
matching_ids_end = []

# Merge dimStartDataRenamed and dimData on the columns to compare
merged_start = dimStartDataRenamed.merge(dimData[cols_to_compare + ['DateKey']], on=cols_to_compare, how='inner')

# Print StartDateKey and Date_id for matching rows
for i, row in merged_start.iterrows():
    #print(f"StartDateKey: {row['StartDateKey']}, DateKey: {row['DateKey']}")
    matching_ids_start.append([row['StartDateKey'], row['DateKey']])


# Merge dimStartDataRenamed and dimData on the columns to compare
merged_end = dimEndDataRenamed.merge(dimData[cols_to_compare + ['DateKey']], on=cols_to_compare, how='inner')

# Print EndDateKey and DateKey for matching rows
for i, row in merged_end.iterrows():
    #print(f"StartDateKey: {row['EndDateKey']}, DateKey: {row['DateKey']}")
    matching_ids_end.append([row['EndDateKey'], row['DateKey']])

In [19]:
# Loop over each pair of StartDateKey and Date_id
for pair in matching_ids_start:
    start_date_id, date_id = pair
    
    # Replace StartDateKey with Date_id in df_desastres
    df_desastres.loc[df_desastres['StartDateKey'] == start_date_id, 'StartDateKey'] = date_id

In [20]:
# Loop over each pair of EndDateKey and Date_id
for pair in matching_ids_end:
    end_date_id, date_id = pair
    
    # Replace EndDateKey with Date_id in df_desastres
    df_desastres.loc[df_desastres['EndDateKey'] == end_date_id, 'EndDateKey'] = date_id

In [21]:
df_desastres

,DisasterID,Year,Seq,Disaster_group,Disaster_subgroup,Disaster_type,Disaster_subtype,Disaster_subsubtype,Event_name,Country,...,Total_affected,Reconstruction_costs,Reconstruction_costs_adjusted,Insured_damages,Insured_damages_adjusted,Total_damages,Total_damages_adjusted,CPI,StartDateKey,EndDateKey
0,1900-9002-CPV,1900,9002,Natural,Climatological,Drought,Drought,NaN,NaN,Cabo Verde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,849084409",0,9815
1,1900-9001-IND,1900,9001,Natural,Climatological,Drought,Drought,NaN,NaN,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,849084409",0,9815
2,1902-0012-GTM,1902,12,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Guatemala,...,NaN,NaN,NaN,NaN,NaN,25000.0,843726.0,"2,963047785",1,9816
3,1902-0003-GTM,1902,3,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,963047785",2,9817
4,1902-0010-GTM,1902,10,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,963047785",3,9818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,2023-0080-ZAF,2023,80,Natural,Hydrological,Flood,NaN,NaN,NaN,South Africa,...,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9812,19120
16564,2023-0110-ZMB,2023,110,Natural,Hydrological,Flood,Flash flood,NaN,NaN,Zambia,...,154608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9807,19219
16565,2023-0068-ZMB,2023,68,Natural,Biological,Epidemic,Bacterial disease,NaN,Cholera,Zambia,...,118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9813,19121
16566,2023-0095-ZWE,2023,95,Natural,Meteorological,Storm,Tropical cyclone,NaN,Tropical cyclone 'Freddy',Zimbabwe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9806,19212


In [22]:
dimStartDataRenamed[cols_to_compare].isin(dimData[cols_to_compare].to_dict(orient='list')).iloc[0]

Local_time    True
Hour          True
Minutes       True
Year          True
Month         True
Day           True
Date          True
Semester      True
Quarter       True
Weekday       True
Season        True
Decade        True
Name: 0, dtype: bool

In [23]:
dimData

,DateKey,Local_time,Hour,Minutes,Year,Month,Day,Date,Semester,Quarter,Weekday,Season,Decade
0,0,NaN,<NA>,<NA>,1900,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,1900s
1,1,20:20,20,20,1902,4,18,1902-04-18,1st Semester,2nd Quarter,Friday,Spring,1900s
2,2,NaN,<NA>,<NA>,1902,4,8,1902-04-08,1st Semester,2nd Quarter,Tuesday,Spring,1900s
3,3,NaN,<NA>,<NA>,1902,10,24,1902-10-24,2nd Semester,4th Quarter,Friday,Fall,1900s
4,4,NaN,<NA>,<NA>,1903,4,29,1903-04-29,1st Semester,2nd Quarter,Wednesday,Spring,1900s
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9401,19216,<NA>,<NA>,<NA>,2023,2,24,2023-02-24,1st Semester,1st Quarter,Friday,Winter,2020s
9402,19217,<NA>,<NA>,<NA>,2023,2,27,2023-02-27,1st Semester,1st Quarter,Monday,Winter,2020s
9403,19218,<NA>,<NA>,<NA>,2023,1,12,2023-01-12,1st Semester,1st Quarter,Thursday,Winter,2020s
9404,19219,<NA>,<NA>,<NA>,2023,3,9,2023-03-09,1st Semester,2nd Quarter,Thursday,Winter,2020s


##### Alterar nan para designação apropriada

In [24]:
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
dimData['MonthName'] = dimData['Month'].map(month_names)

day = {'Sunday': '1', 'Monday': '2', 'Tuesday': '3', 'Wednesday': '4', 'Thursday': '5', 'Friday': '6', 'Saturday': '7'}
dimData['WeekDay_n'] = dimData['Weekday'].map(day).astype(object)

dimData.rename(columns={'Weekday': 'WeekdayName'}, inplace=True)
dimData.rename(columns={'WeekDay_n': 'Weekday'}, inplace=True)


In [25]:
for column in dimData.columns:
    try:
        dimData[column][dimData[column]=="<NA>"] = "Unknown"
        dimData[column][dimData[column].isna()] = "Unknown"
    except:
         dimData[column] = dimData[column].astype(str)
         dimData[column][dimData[column]=="<NA>"] = "Unknown"
         dimData[column][dimData[column].isna()] = "Unknown"

C:\Users\franv\AppData\Local\Temp\ipykernel_12132\604998897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimData[column][dimData[column]=="<NA>"] = "Unknown"
C:\Users\franv\AppData\Local\Temp\ipykernel_12132\604998897.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimData[column][dimData[column].isna()] = "Unknown"
C:\Users\franv\AppData\Local\Temp\ipykernel_12132\604998897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
dimDate = dimData[['DateKey', 'Year', 'Month', 'MonthName', 'Day', 'Decade', 'Semester', 'Season', 'Quarter', 'Weekday', 'WeekdayName', 'Local_time', 'Hour', 'Minutes']]

In [27]:
dimDate

,DateKey,Year,Month,MonthName,Day,Decade,Semester,Season,Quarter,Weekday,WeekdayName,Local_time,Hour,Minutes
0,0,1900,Unknown,Unknown,Unknown,1900s,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,1,1902,4,April,18,1900s,1st Semester,Spring,2nd Quarter,6,Friday,20:20,20,20
2,2,1902,4,April,8,1900s,1st Semester,Spring,2nd Quarter,3,Tuesday,Unknown,Unknown,Unknown
3,3,1902,10,October,24,1900s,2nd Semester,Fall,4th Quarter,6,Friday,Unknown,Unknown,Unknown
4,4,1903,4,April,29,1900s,1st Semester,Spring,2nd Quarter,4,Wednesday,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9401,19216,2023,2,February,24,2020s,1st Semester,Winter,1st Quarter,6,Friday,Unknown,Unknown,Unknown
9402,19217,2023,2,February,27,2020s,1st Semester,Winter,1st Quarter,2,Monday,Unknown,Unknown,Unknown
9403,19218,2023,1,January,12,2020s,1st Semester,Winter,1st Quarter,5,Thursday,Unknown,Unknown,Unknown
9404,19219,2023,3,March,9,2020s,1st Semester,Winter,2nd Quarter,5,Thursday,Unknown,Unknown,Unknown


# Dimensão Type

In [28]:
df_desastres.drop(["Disaster_group"], axis=1, inplace=True)
df_desastres.rename(columns={'Disaster_subgroup': 'Disaster_group'}, inplace=True)

In [29]:
dimType = df_desastres[['Disaster_group', 'Disaster_type', 'Disaster_subtype', 'Disaster_subsubtype']].drop_duplicates()
dimType.insert(0, 'TypeKey', np.arange(len(dimType)))

In [30]:
df_desastres = pd.merge(df_desastres, dimType, on=['Disaster_group', 'Disaster_type', 'Disaster_subtype', 'Disaster_subsubtype'], how='left')
df_desastres = df_desastres.drop(['Disaster_group', 'Disaster_type', 'Disaster_subtype', 'Disaster_subsubtype'], axis=1)

In [31]:
#Valores Null
for column in dimType.columns:
    dimType[column][dimType[column]=="<NA>"] = "Not Applicable"
    dimType[column][dimType[column].isna()] = "Not Applicable"

C:\Users\franv\AppData\Local\Temp\ipykernel_12132\3545949744.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimType[column][dimType[column].isna()] = "Not Applicable"


In [32]:
dimType

,TypeKey,Disaster_group,Disaster_type,Disaster_subtype,Disaster_subsubtype
0,0,Climatological,Drought,Drought,Not Applicable
2,1,Geophysical,Earthquake,Ground movement,Not Applicable
3,2,Geophysical,Volcanic activity,Ash fall,Not Applicable
5,3,Geophysical,Mass movement (dry),Rockfall,Not Applicable
7,4,Meteorological,Storm,Tropical cyclone,Not Applicable
12,5,Hydrological,Flood,Not Applicable,Not Applicable
16,6,Biological,Epidemic,Bacterial disease,Not Applicable
17,7,Geophysical,Mass movement (dry),Landslide,Not Applicable
25,8,Hydrological,Landslide,Avalanche,Not Applicable
33,9,Climatological,Wildfire,Forest fire,Not Applicable


# Dimensão Event

In [33]:
df_desastres.rename(columns={'Event_name': 'EventName','Associated_dis': 'AssociatedDisaster','Associated_dis2': 'AssociatedDisaster2','Mag_value': 'MagnitudeValue','Mag_scale': 'MagnitudeScale', 'OFDA_response': 'OFDA_Response'}, inplace=True)
dimEvent = df_desastres[['EventName','Origin', 'AssociatedDisaster', 'AssociatedDisaster2', 'MagnitudeValue', 'MagnitudeScale', 'OFDA_Response', 'Appeal', 'Declaration', 'CPI']].drop_duplicates()
dimEvent.insert(0, 'EventKey', np.arange(len(dimEvent)))

In [34]:
df_desastres = pd.merge(df_desastres, dimEvent, on=['EventName', 'Origin', 'AssociatedDisaster', 'AssociatedDisaster2', 'MagnitudeValue', 'MagnitudeScale', 'OFDA_Response', 'Appeal', 'Declaration', 'CPI'], how='left')
df_desastres = df_desastres.drop(['EventName', 'Origin', 'AssociatedDisaster', 'AssociatedDisaster2', 'MagnitudeValue', 'MagnitudeScale', 'OFDA_Response', 'Appeal', 'Declaration', 'CPI'], axis=1)

In [35]:
#Missing Values
dimEvent['EventName'].fillna("Name not specified", inplace=True)
dimEvent['Origin'].fillna("Unknown Origin", inplace=True)
dimEvent['AssociatedDisaster'].fillna("No Disaster Associated", inplace=True)
dimEvent['AssociatedDisaster2'].fillna("No Disaster Associated", inplace=True)
dimEvent['MagnitudeValue'].fillna("Unknown", inplace=True)
dimEvent['MagnitudeScale'].fillna("Not Applicable", inplace=True)
dimEvent['OFDA_Response'].fillna("Not Applicable", inplace=True)
dimEvent['Appeal'].fillna("Unknown", inplace=True)
dimEvent['Declaration'].fillna("Unknown", inplace=True)
dimEvent['CPI'].fillna("Not Determined", inplace=True)

In [36]:
dimEvent

,EventKey,EventName,Origin,AssociatedDisaster,AssociatedDisaster2,MagnitudeValue,MagnitudeScale,OFDA_Response,Appeal,Declaration,CPI
0,0,Name not specified,Unknown Origin,Famine,No Disaster Associated,Unknown,Km2,No,No,No,"2,849084409"
1,1,Name not specified,Unknown Origin,No Disaster Associated,No Disaster Associated,Unknown,Km2,No,No,No,"2,849084409"
2,2,Name not specified,Unknown Origin,Tsunami/Tidal wave,No Disaster Associated,8.0,Richter,No,Unknown,Unknown,"2,963047785"
3,3,Santa Maria,Unknown Origin,No Disaster Associated,No Disaster Associated,Unknown,Not Applicable,No,Unknown,Unknown,"2,963047785"
5,4,Name not specified,Unknown Origin,No Disaster Associated,No Disaster Associated,Unknown,Not Applicable,No,Unknown,Unknown,"3,077011162"
...,...,...,...,...,...,...,...,...,...,...,...
16559,8804,Name not specified,Unknown Origin,No Disaster Associated,No Disaster Associated,8.0,Richter,No,Unknown,Yes,Not Determined
16560,8805,Name not specified,Unknown Origin,No Disaster Associated,No Disaster Associated,5.0,Richter,No,Unknown,Unknown,Not Determined
16562,8806,Name not specified,Unknown Origin,No Disaster Associated,No Disaster Associated,145.0,Kph,No,Unknown,Yes,Not Determined
16565,8807,Cholera,Unknown Origin,No Disaster Associated,No Disaster Associated,Unknown,Vaccinated,No,Unknown,Unknown,Not Determined


# Dimensão Location

In [37]:
dimLocation = df_desastres[['Country', 'Country_code', 'Region', 'Continent','Year']].drop_duplicates()
dimLocation

,Country,Country_code,Region,Continent,Year
0,Cabo Verde,CPV,Western Africa,Africa,1900
1,India,IND,Southern Asia,Asia,1900
2,Guatemala,GTM,Central America,Americas,1902
5,Canada,CAN,Northern America,Americas,1903
6,Comoros,COM,Eastern Africa,Africa,1903
...,...,...,...,...,...
16556,Namibia,NAM,Southern Africa,Africa,2023
16559,Syrian Arab Republic,SYR,Western Asia,Asia,2023
16563,South Africa,ZAF,Southern Africa,Africa,2023
16566,Zimbabwe,ZWE,Eastern Africa,Africa,2023


In [38]:
dfaux = df_desastres[['Country', 'Country_code', 'Region', 'Continent']]
dfaux

,Country,Country_code,Region,Continent
0,Cabo Verde,CPV,Western Africa,Africa
1,India,IND,Southern Asia,Asia
2,Guatemala,GTM,Central America,Americas
3,Guatemala,GTM,Central America,Americas
4,Guatemala,GTM,Central America,Americas
...,...,...,...,...
16563,South Africa,ZAF,Southern Africa,Africa
16564,Zambia,ZMB,Eastern Africa,Africa
16565,Zambia,ZMB,Eastern Africa,Africa
16566,Zimbabwe,ZWE,Eastern Africa,Africa


In [39]:
# Transform Years from columns to rows
df_GDP_melted = df_GDP.melt(id_vars=['Country', 'Country_code'], var_name='Year', value_name='GDP')
df_turismo_melted = df_turismo.melt(id_vars=['Country', 'Country_code'], var_name='Year', value_name='Turismo')
df_desemprego_melted = df_desemprego.melt(id_vars=['Country', 'Country_code'], var_name='Year', value_name='Desemprego')
df_populacao_melted = df_populacao.melt(id_vars=['Country', 'Country_code'], var_name='Year', value_name='Populacao')

# Merge the datasets
df_stats = pd.merge(df_GDP_melted, df_turismo_melted, on=['Country', 'Country_code', 'Year'], how='outer')
df_stats = pd.merge(df_stats, df_desemprego_melted, on=['Country', 'Country_code', 'Year'], how='outer')
df_stats = pd.merge(df_stats, df_populacao_melted, on=['Country', 'Country_code', 'Year'], how='outer')

# sort the merged dataframe by country and year
df_stats = df_stats.sort_values(['Country', 'Year'])

In [40]:
# Change column type
df_stats['Year'] = pd.to_numeric(df_stats['Year'], errors='coerce').astype('Int64').fillna(pd.NA)
df_stats['GDP'] = pd.to_numeric(df_stats['GDP'].str.replace(',', '.'), errors='coerce')
df_stats['Desemprego'] = pd.to_numeric(df_stats['Desemprego'].str.replace(',', '.'), errors='coerce')

In [41]:
df_agricultura = df_agricultura.pivot_table(index=['Country_code', 'Country', 'Year'], columns='Item', values='Value')

df_agricultura.drop(["Agriculture","Cereals, primary","Crops","Livestock","Meat indigenous, total","Milk, Total","Roots and Tubers, Total","Sugar Crops Primary","Vegetables and Fruit Primary"], axis=1, inplace=True)
df_stats = pd.merge(df_stats, df_agricultura, on=['Country', 'Country_code', 'Year'], how='left')
df_stats

,Country,Country_code,Year,GDP,Turismo,Desemprego,Populacao,Food,Non Food
0,Afghanistan,AFG,1960,5.377778e+08,NaN,NaN,8622466.0,NaN,NaN
1,Afghanistan,AFG,1961,5.488889e+08,NaN,NaN,8790140.0,41.48,161.94
2,Afghanistan,AFG,1962,5.466667e+08,NaN,NaN,8969047.0,41.81,163.88
3,Afghanistan,AFG,1963,7.511112e+08,NaN,NaN,9157465.0,41.62,166.39
4,Afghanistan,AFG,1964,8.000000e+08,NaN,NaN,9355514.0,45.13,170.78
...,...,...,...,...,...,...,...,...,...
16591,Zimbabwe,ZWE,2017,1.758489e+10,2423000.0,6.279,14751101.0,106.55,74.55
16592,Zimbabwe,ZWE,2018,3.415607e+10,2580000.0,6.784,15052184.0,108.86,130.20
16593,Zimbabwe,ZWE,2019,2.183223e+10,2294000.0,7.370,15354608.0,105.55,105.94
16594,Zimbabwe,ZWE,2020,2.150970e+10,639000.0,7.898,15669666.0,109.60,113.73


In [42]:
df_stats = pd.merge(df_stats, dfaux, on=['Country', 'Country_code'], how="outer")
df_stats.drop_duplicates(inplace=True)

In [43]:
df_stats['Year'] = df_stats['Year'].astype(object)

In [44]:
dimLocation = pd.merge(dimLocation, df_stats, on=['Country', 'Country_code','Region','Continent','Year'], how='outer')
dimLocation.drop_duplicates(inplace=True)

C:\Users\franv\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [45]:
dimLocation = dimLocation.dropna(subset=['Region'])

In [46]:
dimLocation.insert(0, 'LocationKey', np.arange(len(dimLocation)))

In [47]:
dimLocation['Year'] = pd.to_numeric(dimLocation['Year'], errors='coerce').astype('Int64').fillna(pd.NA)

In [48]:
df_desastres['Year'] = df_desastres['Year'].astype(str)
dimLocation['Year'] = dimLocation['Year'].astype(str)

In [49]:
# Merge the two dataframes to put the respective LocationKey for each disaster
df_desastres = pd.merge(df_desastres, dimLocation, on=['Country', 'Country_code', 'Region', 'Continent','Year'], how='left')
df_desastres = df_desastres.drop_duplicates()

In [50]:
dimLocation['RowEffectiveDate'] = (pd.to_datetime(dimLocation['Year'], format='%Y', errors='coerce') + pd.offsets.YearBegin(0)).dt.date
dimLocation['RowExpirationDate'] = (pd.to_datetime(dimLocation['Year'], format='%Y', errors='coerce') + pd.offsets.YearEnd(0)).dt.date

dimLocation['Year'] = pd.to_datetime(dimLocation['Year'], format='%Y', errors='coerce')
dimLocation = dimLocation.sort_values(['Country', 'Year'])
dimLocation['CurrentRowIndicator'] = dimLocation.groupby('Country')['Year'].rank(ascending=False).eq(1).map({True: 'current', False: 'expired'})

dimLocation.dropna(subset=['RowEffectiveDate'], inplace=True)
dimLocation.drop(["Year"], axis=1, inplace=True)

In [51]:
columnRename = {
    'GDP': 'CountryGDP',
    'Turismo': 'CountryTourism',
    'Desemprego': 'CountryUnemployment',
    'Populacao': 'CountryPopulation',
    'Food': 'CountryAgricultureFood',
    'Non Food': 'CountryAgricultureNonFood'
}

dimLocation.rename(columns=columnRename, inplace=True)

In [52]:
for column in dimLocation.columns:
    try:
        dimLocation[column][dimLocation[column]=="<NA>"] = "Unknown"
        dimLocation[column][dimLocation[column].isna()] = "Unknown"
    except:
        dimLocation[column] = dimLocation[column].astype(str)
        dimLocation[column][dimLocation[column]=="<NA>"] = "Unknown"
        dimLocation[column][dimLocation[column].isna()] = "Unknown"


C:\Users\franv\AppData\Local\Temp\ipykernel_12132\326436091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimLocation[column][dimLocation[column]=="<NA>"] = "Unknown"
C:\Users\franv\AppData\Local\Temp\ipykernel_12132\326436091.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimLocation[column][dimLocation[column].isna()] = "Unknown"
C:\Users\franv\AppData\Local\Temp\ipykernel_12132\326436091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

C:\Users\franv\AppData\Local\Temp\ipykernel_12132\326436091.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimLocation[column][dimLocation[column].isna()] = "Unknown"
C:\Users\franv\AppData\Local\Temp\ipykernel_12132\326436091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimLocation[column][dimLocation[column]=="<NA>"] = "Unknown"
C:\Users\franv\AppData\Local\Temp\ipykernel_12132\326436091.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [53]:
dimLocation

,LocationKey,Country,Country_code,Region,Continent,CountryGDP,CountryTourism,CountryUnemployment,CountryPopulation,CountryAgricultureFood,CountryAgricultureNonFood,RowEffectiveDate,RowExpirationDate,CurrentRowIndicator
199,199,Afghanistan,AFG,Southern Asia,Asia,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1954-01-01,1954-12-31,expired
220,220,Afghanistan,AFG,Southern Asia,Asia,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1956-01-01,1956-12-31,expired
6146,6146,Afghanistan,AFG,Southern Asia,Asia,537777811.1,Unknown,Unknown,8622466.0,Unknown,Unknown,1960-01-01,1960-12-31,expired
6147,6147,Afghanistan,AFG,Southern Asia,Asia,548888895.6,Unknown,Unknown,8790140.0,41.48,161.94,1961-01-01,1961-12-31,expired
6148,6148,Afghanistan,AFG,Southern Asia,Asia,546666677.8,Unknown,Unknown,8969047.0,41.81,163.88,1962-01-01,1962-12-31,expired
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5618,5618,Zimbabwe,ZWE,Eastern Africa,Africa,21832234926.0,2294000.0,7.37,15354608.0,105.55,105.94,2019-01-01,2019-12-31,expired
17613,13417,Zimbabwe,ZWE,Eastern Africa,Africa,21509698406.0,639000.0,7.898,15669666.0,109.6,113.73,2020-01-01,2020-12-31,expired
6012,6012,Zimbabwe,ZWE,Eastern Africa,Africa,28371238666.0,Unknown,8.067,15993524.0,132.04,96.36,2021-01-01,2021-12-31,expired
6060,6060,Zimbabwe,ZWE,Eastern Africa,Africa,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,2022-01-01,2022-12-31,expired


# Fact Disasters

In [54]:
factDisaster = df_desastres[["DisasterID","TypeKey","LocationKey","StartDateKey","EndDateKey","EventKey",
    "Total_deaths","N_injured","N_affected","N_homeless","Total_affected","AID_contribution","Reconstruction_costs","Reconstruction_costs_adjusted",
    "Insured_damages","Insured_damages_adjusted","Total_damages"]]
factDisaster

,DisasterID,TypeKey,LocationKey,StartDateKey,EndDateKey,EventKey,Total_deaths,N_injured,N_affected,N_homeless,Total_affected,AID_contribution,Reconstruction_costs,Reconstruction_costs_adjusted,Insured_damages,Insured_damages_adjusted,Total_damages
0,1900-9002-CPV,0,0,0,9815,0,11000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1900-9001-IND,0,1,0,9815,1,1250000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1902-0012-GTM,1,2,1,9816,2,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25000.0
3,1902-0003-GTM,2,2,2,9817,3,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1902-0010-GTM,2,2,3,9818,3,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,2023-0080-ZAF,5,6143,9812,19120,8803,17.0,NaN,1000.0,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,NaN
16564,2023-0110-ZMB,27,6137,9807,19219,8800,NaN,NaN,154608.0,NaN,154608.0,NaN,NaN,NaN,NaN,NaN,NaN
16565,2023-0068-ZMB,6,6137,9813,19121,8807,2.0,118.0,NaN,NaN,118.0,NaN,NaN,NaN,NaN,NaN,NaN
16566,2023-0095-ZWE,4,6144,9806,19212,8801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
factDisaster['Total_deaths'] = pd.to_numeric(factDisaster['Total_deaths'], errors='coerce').astype('Int64')
factDisaster['N_injured'] = pd.to_numeric(factDisaster['N_injured'], errors='coerce').astype('Int64')
factDisaster['N_affected'] = pd.to_numeric(factDisaster['N_affected'], errors='coerce').astype('Int64')
factDisaster['N_homeless'] = pd.to_numeric(factDisaster['N_homeless'], errors='coerce').astype('Int64')
factDisaster['Total_affected'] = pd.to_numeric(factDisaster['Total_affected'], errors='coerce').astype('Int64')

C:\Users\franv\AppData\Local\Temp\ipykernel_12132\1209632092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factDisaster['Total_deaths'] = pd.to_numeric(factDisaster['Total_deaths'], errors='coerce').astype('Int64')
C:\Users\franv\AppData\Local\Temp\ipykernel_12132\1209632092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factDisaster['N_injured'] = pd.to_numeric(factDisaster['N_injured'], errors='coerce').astype('Int64')
C:\Users\franv\AppData\Local\Temp\ipykernel_12132\1209632092.py:3: SettingWi

In [56]:
factDisaster

,DisasterID,TypeKey,LocationKey,StartDateKey,EndDateKey,EventKey,Total_deaths,N_injured,N_affected,N_homeless,Total_affected,AID_contribution,Reconstruction_costs,Reconstruction_costs_adjusted,Insured_damages,Insured_damages_adjusted,Total_damages
0,1900-9002-CPV,0,0,0,9815,0,11000,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
1,1900-9001-IND,0,1,0,9815,1,1250000,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
2,1902-0012-GTM,1,2,1,9816,2,2000,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,25000.0
3,1902-0003-GTM,2,2,2,9817,3,1000,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
4,1902-0010-GTM,2,2,3,9818,3,6000,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,2023-0080-ZAF,5,6143,9812,19120,8803,17,<NA>,1000,<NA>,1000,NaN,NaN,NaN,NaN,NaN,NaN
16564,2023-0110-ZMB,27,6137,9807,19219,8800,<NA>,<NA>,154608,<NA>,154608,NaN,NaN,NaN,NaN,NaN,NaN
16565,2023-0068-ZMB,6,6137,9813,19121,8807,2,118,<NA>,<NA>,118,NaN,NaN,NaN,NaN,NaN,NaN
16566,2023-0095-ZWE,4,6144,9806,19212,8801,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN


# Extract CSV'S

In [57]:
#Extract the files
dimDate.to_csv('dimDate.csv', index=False)
dimType.to_csv('dimType.csv', index=False)
dimEvent.to_csv('dimEvent.csv', index=False)
dimLocation.to_csv('dimLocation.csv', index=False)
factDisaster.to_csv('factDisaster.csv', index=False)